In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./data/data_df.csv")
data_df = df.copy()
data_df = data_df.astype({"WDT_LEN" : "str", "WGT_WGT" : "str", "IND_CD" : "str", "OUD_LEN" : "str", "STL_CD" : "str"})

kind_of_ann = data_df["PNSPRC_CD"].unique().tolist()
kind_of_cycle = data_df["cycle"].unique().tolist()

print(kind_of_ann)
print(kind_of_cycle)

['AN11', 'AN10', 'AN12', 'AN13']
['620', '750', '725', '700', '725(표준)', '작업지시서 소둔 Cycle 적용', '700(표준)', '735', '720', '700*Max', '725*Max', '750*Max']


In [12]:
# for i in range(len(kind_of_ann)):
#     for j in range(len(kind_of_cycle)): 
#         separated_df = data_df.loc[(data_df['PNSPRC_CD'] == kind_of_ann[i]) & (data_df['cycle'] == kind_of_cycle[j])]
#         if separated_df.shape[0] >= 1 : print(separated_df.shape[0])
#         # ann차수, cycle을 통한 여러개의 조합들 중, 해당하는 열이 없는 조합일 때는 그냥 스킵만 해줄까 ?


# 첫번째 그룹
separated_df = data_df.loc[(data_df['PNSPRC_CD'] == 'AN11') & (data_df['cycle'] == '620')]
separated_df
# separated_df.tail()

,Unnamed: 0,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,4,09KA1B77311,SCP1-H-B,AN11,620,309.0,4811,508,1668,10021,1282N,NaN
19,24,22E11E19111,S20CM-R,AN11,620,193.2,1555,508,1250,10021,1282N,NaN
53,58,22G11A03111,S20CM-S,AN11,620,154.0,1414,508,1322,10021,1282N,NaN
54,59,22G11A03112,S20CM-S,AN11,620,154.0,1447,508,1335,10021,1282N,NaN
56,61,22G11L69111,S15CM-S,AN11,620,171.0,1304,508,1223,10021,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
212,217,22H11P52111,S20CM-R,AN11,620,186.0,1747,508,1335,10021,1282N,NaN
213,218,22H11P52112,S20CM-R,AN11,620,186.0,1702,508,1320,10021,1282N,NaN
368,373,22J11015111,S20CM-S,AN11,620,1231.0,22970,610,1843,10021,1282N,NaN
369,374,22J11027111,SAE1010M-R,AN11,620,1000.0,18310,610,1828,10021,1282N,NaN


In [5]:
coil_number = separated_df["COIL_NO"].tolist()
ann_number = separated_df["PNSPRC_CD"].tolist()
cycle = separated_df["cycle"].tolist()
coil_length = separated_df["WDT_LEN"].tolist()
coil_weight = separated_df["WGT_WGT"].tolist()
coil_inner = separated_df["IND_CD"].tolist()
coil_outer = separated_df["OUD_LEN"].tolist()
coil_emergency = separated_df["EMG_CD"].tolist()

In [7]:
base_capacity_information = pd.read_csv("./data/base_capacity_information.csv")
base_capacity_information.head()
base_capacity_information.set

,Maker,Base_number,Weight(Ton),Height(mm),Outer_max,Outer_min,Inner
0,EBNER,1,35,2800,1700,900,508
1,EBNER,2,35,2800,1700,900,508
2,EBNER,3,35,2800,1700,900,508
3,EBNER,4,35,2800,1700,900,508
4,EBNER,5,35,2800,1700,900,508


In [10]:
base_capacity_information


first_base_group = base_capacity_information.loc[(base_capacity_information['Maker'] == 'EBNER') & (base_capacity_information['Base_number'] <= 6)]
first_base_group

,Maker,Base_number,Weight(Ton),Height(mm),Outer_max,Outer_min,Inner
0,EBNER,1,35,2800,1700,900,508
1,EBNER,2,35,2800,1700,900,508
2,EBNER,3,35,2800,1700,900,508
3,EBNER,4,35,2800,1700,900,508
4,EBNER,5,35,2800,1700,900,508
5,EBNER,6,35,2800,1700,900,508
